# Исследование музыкальных предпочтений жителей двух столиц

**Описание проекта:**     
Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных сервиса Яндекс.Музыки мы сравним поведение пользователей двух столиц.

**Цель исследования** — проверить три гипотезы:  
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.  
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.   
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.  

**Ход исследования:**  
Данные о поведении пользователей хранятся в файле `yandex_music_project.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Я проверю данные на ошибки и оценю их влияние на исследование. Затем, на этапе предобработки я поищу возможность исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных
 2. Предобработка данных
 3. Проверка гипотез

## Обзор данных

**1.1 Откроем файл с данными и сохраним его в переменную `df`:**

In [1]:
# импорт библиотеки pandas
import pandas as pd

# чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/yandex_music_project.csv')

**1.2 Выведем первые 10 строк датафрейма `df`:**

In [2]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


**1.3 Выведем основную информацию о датафрейме с помощью метода `info()`:**

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


**Вывод:** Датафрейм **`df`** содержит 65079 строк и 7 столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели. 

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

### Стиль заголовков

#### Отобразим названия всех столбцов:

In [4]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Приведём названия в соответствие с хорошим стилем:  
- устраним пробелы  
- сделаем все символы строчными  
- несколько слов в названии запишем в «змеином_регистре»

In [7]:
# переименование столбцов
df = df.rename(columns={'userID':'user_id', 'Track':'track', 'City':'city', 'Day':'day'}) 

# проверка результатов
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Работа с пропусками

#### Посчитаем, сколько в таблице пропущенных значений:

In [8]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование. Так, пропуски в столбцах `track` и `artist`  не важны для нашего исследования. Достаточно будет заменить их явными обозначениями.

Однако пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Но такой возможности, к сожалению, нет, поэтому предпримем следующие действия:
* заполним эти пропуски также явными обозначениями
* оценим, насколько они повредят расчётам

In [9]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
for columns in columns_to_replace:
    df[columns] = df[columns].fillna('unknown')

Убедимся, что в таблице не осталось пропусков:

In [10]:
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Работа с дубликатами

Посчитаем явные дубликаты в таблице:

In [11]:
# подсчёт явных дубликатов
df.duplicated().sum() 

3826

Удалим явные дубликаты методом **drop_duplicates():**

In [15]:
# удаление дубликатов
df = df.drop_duplicates()

Убедимся, что полностью избавились от явных дубликатов:

In [16]:
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

Теперь перейдем к работе с неявными дубликатами.  
#### Проверим уникальные названия жанров в колонке `genre`. 
Выведем их отсортированными в алфавитном порядке:

In [17]:
# Просмотр уникальных названий жанров
df_sorted_genre = df['genre'].sort_values().unique()
print(df_sorted_genre)

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

Среди уникальных наименований обнаружены следующие неявные дубликаты жанра `hiphop`:
* *hip*,
* *hop*,
* *hip-hop*.

#### Очистим от них таблицу с помощью метода replace():

In [18]:
# Устранение неявных дубликатов
duplicates = ['hip', 'hip-hop', 'hop']
genre = 'hiphop'
df['genre'] = df['genre'].replace(duplicates, genre)

#### Убедимся, что неявные дубликаты жанра `hiphop` устранены:

In [19]:
# Проверка на неявные дубликаты
print(df['genre'].sort_values().unique())

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

**Выводы:**

На этапе предобработки были обнаружены три проблемы в данных:
- нарушения в стиле заголовков
- пропущенные значения
- дубликаты — явные и неявные

Были исправлены заголовки, чтобы упростить работу с таблицей.  
Пропуски были заменены на значение`'unknown'`. Также были устранены явные и неявные дубликаты.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга  
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу

#### Оценим активность пользователей в каждом городе:

In [25]:
# Подсчёт прослушиваний в каждом городе
activity_by_city = df.groupby('city')['user_id'].count()
activity_by_city

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Это может быть связано с тем, что просто самих пользователей в Москве больше, чем в Санкт-Петербурге.  

#### Сгруппируем данные по дню недели и посчитаем прослушивания в понедельник, среду и пятницу:

*Будем учитывать, что в данных есть информация о прослушиваниях только за эти дни недели*

In [24]:
# Подсчёт прослушиваний в каждый из трёх дней
activity_by_day = df.groupby('day')['user_id'].count()
activity_by_day

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

#### Создадим функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города:

In [67]:
def number_tracks(day, city):
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city] 
    track_list_count = track_list['user_id'].count()
    return track_list_count

Вызовем функцию `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней:

In [27]:
# количество прослушиваний в Москве по понедельникам 
number_tracks('Monday','Moscow')

15740

In [28]:
# количество прослушиваний в Санкт-Петербурге по понедельникам
number_tracks('Monday','Saint-Petersburg')

5614

In [29]:
# количество прослушиваний в Москве по средам
number_tracks('Wednesday','Moscow')

11056

In [30]:
# количество прослушиваний в Санкт-Петербурге по средам
number_tracks('Wednesday','Saint-Petersburg')

7003

In [31]:
# количество прослушиваний в Москве по пятницам
number_tracks('Friday','Moscow')

15945

In [33]:
# количество прослушиваний в Санкт-Петербурге по пятницам
number_tracks('Friday','Saint-Petersburg')

5895

#### Создадим таблицу, где будут отражены данные по каждому городу и дню недели:

In [34]:
# Таблица с результатами
data = [['Moscow', 15740, 11056, 15945],  
        ['Saint-Petersburg', 5614, 7003, 5895]]
     
columns = ['city', 'monday', 'wednesday', 'friday']

pd.DataFrame(data=data, columns=['city', 'monday', 'wednesday', 'friday'])

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним отдельные таблицы по каждому городу в разные переменные:
* по Москве — в `moscow_general`
* по Санкт-Петербургу — в `spb_general`

In [36]:
# получение таблицы moscow_general
moscow_general = df[df['city'] == 'Moscow']

# получение таблицы spb_general
spb_general = df[df['city'] == 'Saint-Petersburg']

Создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными
* день недели
* начальная временная метка в формате 'hh:mm' 
* последняя временная метка в формате 'hh:mm'

Функция вернет информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени:

In [38]:
def genre_weekday(df, day, time1, time2):
    # оставляем в genre_df только те строки df, у которых день равен day
    genre_df = df[df['day'] == day] 
    # оставляем в genre_df только те строки genre_df, у которых время меньше time2
    genre_df = genre_df[genre_df['time'] > time1] 
    # оставляем в genre_df только те строки genre_df, у которых время больше time1
    genre_df = genre_df[genre_df['time'] < time2] 
    # сгруппируем отфильтрованный датафрейм по столбцу с названиями жанров, возьмём столбец genre и посчитаем кол-во строк для каждого жанра методом count()
    genre_df_grouped = genre_df.groupby('genre')['genre'].count() 
    # отсортируем результат по убыванию (чтобы в начале Series оказались самые популярные жанры)
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False) 
    # вернём Series с 10 самыми популярными жанрами в указанный отрезок времени заданного дня
    return genre_df_sorted[:10]

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [39]:
# вызов функции для утра понедельника в Москве
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [40]:
# вызов функции для утра понедельника в Петербурге
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [41]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [42]:
# вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы:**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  
Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

#### Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра.
Сохраним результат в таблицу `moscow_genres` и отсортируем значения в порядке убывания:

In [49]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)

#### Выведем первые десять строк `moscow_genres`:

In [46]:
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Повторим те же действия и для Петербурга.  
#### Сгруппируем таблицу `spb_general` по жанру и посчитаем прослушивания треков каждого жанра.   
Сохраним результат в таблицу `spb_genres` и отсортируем значенияв порядке убывания:

In [50]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

#### Выведем первые десять строк `spb_genres`:

In [48]:
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы:**   
Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

В ходе проверки трех гипотез было установлено следующее:

1. День недели ***по-разному*** влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения ***не сильно*** меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга ***больше общего*** чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.